In [ ]:
"""
just get synapse counts for root id
get synapses in bounding box
linkbuilder (generalize)
ID rough spot checker
"""

In [5]:
def get_synapse_counts(root_ids, datastack, cleft_thresh=0):
    """Get synapse counts for a list of root IDs.
    
    Arguments:
    root_ids -- a list of root IDs to get synapse counts for (list of int or str, will also accept a single int or str)
    datastack -- the name of the datastack the IDs are from (str)
    cleft_thresh -- the cleft score bleow which to exclude synapses, currently only works with "flywire_fafb_production" datastack (int, default 0)
    
    Returns:
    synapse_dict -- a dictionary containing the requested synapse counts
    """

    # returns error if cleft thresholding is used with non-flywire datasets #
    if datastack != "flywire_fafb_production" and cleft_thresh != 0:
        raise ValueError("Cleft thresholding currently only works for the 'flywire_fafb_production' dataset.")

    ### CURRENTLY ASSUMES CLEFT SCORE COLUMN NAME IS "cleft_score", ONLY WORKS WITH FLYWIRE ###
    cleft_score_column_name = "cleft_score"

    # sets CAVE client object using datastack name #
    client = CAVEclient(datastack_name=datastack)

    # gets metadata for chosen datastack as dict #
    stack_info = client.info.get_datastack_info()

    # gets name of synapse table using stack_info #
    synapse_table_name = stack_info["synapse_table"]

    # converts root IDs to list of integers for passing into query_table method #
    root_ids = list(map(int, root_ids))

    # creates empty synapse dict to fill with counts #
    synapse_dict = {}

    # iterates over root IDs to get synapse counts #
    for root_id in root_ids:
        in_df = client.materialize.query_table(
                synapse_table_name,
                filter_in_dict={"post_pt_root_id": [root_id]},
            )
        out_df = client.materialize.query_table(
                synapse_table_name,
                filter_in_dict={"pre_pt_root_id": [root_id]},
            )
        
        # drops synapses if asked #
        if cleft_thresh > 0:
            in_df = in_df[
                in_df[cleft_score_column_name] >= float(cleft_thresh)
            ].reset_index(drop=True)
            out_df = out_df[
                out_df[cleft_score_column_name] >= float(cleft_thresh)
            ].reset_index(drop=True)

        # gets synapse counts by counting length of dfs #
        incoming = len(in_df)
        outgoing = len(out_df)

        # adds values to synapse dict #
        synapse_dict[str(root_id)] = {
            "incoming" : incoming,
            "outgoing" : outgoing,
            "total" : incoming + outgoing,
        }
    
    return synapse_dict

In [6]:
get_synapse_counts([720575941498482043, 720575941619873407, 720575941628378338],"brain_and_nerve_cord")

{'720575941498482043': {'incoming': 2001, 'outgoing': 12520, 'total': 14521},
 '720575941619873407': {'incoming': 7911, 'outgoing': 26319, 'total': 34230},
 '720575941628378338': {'incoming': 323, 'outgoing': 551, 'total': 874}}

In [ ]:
# REBUILD PROCEDURE #

# git pull ; pip uninstall tracer_tools ; python -m pip install -e "F:\Archive Backup\Jay\Work Related\Work Apps\tracer_tools"

# FOR LAPTOP #
# git pull ; pip uninstall tracer_tools ; python -m pip install -e "C:\Users\jgager\coding\tracer_tools"


# python -m build;pip uninstall tracer_tools;python -m pip install -e "F:\Archive Backup\Jay\Work Related\Work Apps\tracer_tools"

# python -m build
# pip uninstall tracer_tools
# python -m pip install -e "F:\Archive Backup\Jay\Work Related\Work Apps\tracer_tools"

In [1]:
import cloudvolume
from caveclient import CAVEclient
import pandas as pd
from nglui.statebuilder import *
import json
from osteoid import Skeleton
import numpy as np
import microviewer
import time
import sys
import plotly.graph_objects as go
import statistics

import tracer_tools as tt
from pprint import pprint

In [ ]:
bbox_coord_list = [[[1,2,3],[4,5,6]],[[7,8,9],[1,2,3]]]    
nm_bbox_list = [[tt.convert_coord_res(point,res_current=[4,4,40]) for point in bbox] for bbox in bbox_coord_list]
print(nm_bbox_list)

In [ ]:
def calc_distance(point_a, point_b, xyz_resolution):
    """Calculate distance in 3D between two points in nanometers based on the viewer resolution.
    
    Arguments:
    point_a -- xyz coordinates of first point (list of ints)
    point_b -- xyz coordinates of second point (list of ints)
    xyz_resolution -- the nanometers per voxel in the x, y, and z directions (list of ints)

    Returns:
    dist -- the 3D distance in nanometers between points a and b"""

    # calculates distance in x, y, and z dimensions #
    xdist = (xyz_resolution[0] * point_a[0]) - (xyz_resolution[0] * point_b[0])
    ydist = (xyz_resolution[1] * point_a[1]) - (xyz_resolution[1] * point_b[1])
    zdist = (xyz_resolution[2] * point_a[2]) - (xyz_resolution[2] * point_b[2])

    # uses distance formula to calculate distance in 3D #
    dist = ((xdist ** 2) + (ydist ** 2) + (zdist ** 2)) ** 0.5

    return dist

In [ ]:
calc_distance([145983, 59737, 3304],[147352, 59765, 3184],[4,4,40])

In [ ]:
tt.get_stack_data("brain_and_nerve_cord")

In [ ]:
def get_datastack_config(datastack):
    """Get relevant information for a given datastack"""
    
    # sets CAVE client object using datastack name #
    client = CAVEclient(datastack_name=datastack)

    # gets metadata for chosen datastack as dict #
    stack_info = client.info.get_datastack_info()
    
    pprint(stack_info)

    if datastack == "flywire_fafb_production":
        config = {
            "datastack_name" : datastack,
            "em_url":stack_info["aligned_volume"]["image_source"],
            "segmentation_url":stack_info["segmentation_source"],
            "viewer_resolution":[
                stack_info["viewer_resolution_x"],
                stack_info["viewer_resolution_y"],
                stack_info["viewer_resolution_z"],   
            ],
            "synapse_table_name" : stack_info["synapse_table"],
            "incoming_syn_col_name":"post_pt_root_id",
            "outgoing_syn_col_name":"pre_pt_root_id",
            "nucleus_table":stack_info["soma_table"],
            "skeleton_url":stack_info["skeleton_source"],
            "recommended_syn_cleft_thresh":50,
        }


In [ ]:
get_datastack_config("flywire_fafb_production")

In [ ]:
# [out_nt, in_dict, in_plot] = get_nt(720575940651305718, "flywire_fafb_production", incoming=True)
# print(out_nt)
# pprint(in_dict)
# in_plot.show()

nt_list = tt.get_nt([720575940651305718,720575940626405690,720575940618680550], "flywire_fafb_production")
print(nt_list)

In [ ]:
def roots_to_nt_link(root_ids, datastack):
    """Generate a neuroglancer link from a list of root IDs color coded by dominant outgoing synapse neurotransmitter. CURRENTLY ONLY WORKS WITH FLYWIRE
    
    Arguments:
    root_ids -- a list of root IDs (list of int or str)
    datastack -- the name of the datastack the root IDs are from (str)

    Returns:
    link -- a neuroglancer url with nt-color-coded segments (str)
    """

    # gets dominant outgoing neurotransmitters for list of root IDs #
    nts = tt.get_nt(root_ids, datastack)

    # creates empty list to fill with nt-paired hex values for color coding #
    color_list = []

    # adds hex values to color list based on dominant nt for each ID #
    for nt in nts:
        if nt == "ach":
            color_list.append("#ff4958")
        elif nt == "da":
            color_list.append("#ff944d")
        elif nt == "gaba":
            color_list.append("#e7d84a")
        elif nt == "glut":
            color_list.append("#44d44b")
        elif nt == "oct":
            color_list.append("#0084ff")
        elif nt == "ser":
            color_list.append("#b65eff")

    # builds neuroglancer link using root IDs and list of custom colors #
    link = tt.build_ng_link(root_ids, datastack, custom_colors=color_list)

    return link

In [ ]:
roots_to_nt_link([720575940625342472,720575940627818499,720575940631449916,720575940651305718,720575940618680550,720575940626405690], "flywire_fafb_production")

In [ ]:
##################################### ROOT ID ROUGH SPOT CHECKER ########################################

In [ ]:
def build_ng_link(
    root_ids,
    datastack,
    incoming=False,
    outgoing=False,
    cleft_thresh=0,
    white=False,
    custom_colors=False,
    bbox_coord_list=False
):
    """Build a neuroglancer state url from a list of root IDs.

    Arguments:
    root_ids -- a list of 18-digit root IDs (list of str)
    datastack -- the name of the datastack the root IDs belong to (str)
    incoming -- whether to include incoming synapses (bool, default False)
    outgoing -- whether to include outgoing synapses (bool, default False)
    cleft_thresh -- the cleft score threshold below which to exclude synapses, currently only works for flywire (int, default 0)
    white -- whether or not to make all the segment colors white (bool, default False)
    custom_colors -- if a list of hex values is passed they will be used to color the neurons in the same order as the root_ids list (list of str, default False)
    bbox_coord_list -- option to add one or more bounding boxes to the final link by inputting their corner coordinates, e.g. for two [[[147355, 59771, 3184],[148090, 60226, 3064]],[[147883, 60485, 3064],[148460, 61040, 2964]]] (list of lists of lists of ints)
    
    Returns:
    ng_url -- the url for the constructed neuroglancer state (str)
    """

    ### CURRENTLY ASSUMES ROOT COLUMN NAMES ARE "pre/post_pt_root_id" ###
    ### CURRENTLY ASSUMES COORD COLUMN NAMES ARE "pre/post_pt_position" ###

    ### CURRENTLY ASSUMES CLEFT SCORE COLUMN NAME IS "cleft_score", ONLY WORKS WITH FLYWIRE ###
    cleft_score_column_name = "cleft_score"

    # sets CAVE client object using datastack name #
    client = CAVEclient(datastack_name=datastack)

    # gets metadata for chosen datastack as dict #
    stack_info = client.info.get_datastack_info()

    # gets base url from stack info #
    base_url = stack_info["viewer_site"]

    # gets viewer resolution from stack info #
    viewer_res = [
        stack_info["viewer_resolution_x"],
        stack_info["viewer_resolution_y"],
        stack_info["viewer_resolution_z"],
    ]

    # if synapses are requested, sets name of synapse table using metadata #
    if incoming == True or outgoing == True:
        synapse_table_name = stack_info["synapse_table"]

    # determines names of synapse table columns and ng tabs based on arguments #
    # also sets syn_state variable for use later #
    if incoming == True and outgoing == False:
        syn_state = "in"
        syn_col_name_1 = "post_pt_root_id"
        syn_tab_name_1 = "Incoming Synapses"
    elif outgoing == True and incoming == False:
        syn_state = "out"
        syn_col_name_1 = "pre_pt_root_id"
        syn_tab_name_1 = "Outgoing Synapses"
    elif incoming == True and outgoing == True:
        syn_state = "both"
        syn_col_name_1 = "post_pt_root_id"
        syn_tab_name_1 = "Incoming Synapses"
        syn_col_name_2 = "pre_pt_root_id"
        syn_tab_name_2 = "Outgoing Synapses"
    else:
        syn_state = "none"

    # converts root IDs to integers for passing into query_table method #
    root_ids = list(map(int, root_ids))

    # creates empty list to populate with annotation dfs #
    anno_df_list = []

    # queries synapse table based on arguments #
    if syn_state != "none":
        syn_df_1 = client.materialize.query_table(
            synapse_table_name,
            filter_in_dict={syn_col_name_1: root_ids},
        )

        # removes synapses with cleft scores below threshold #
        if cleft_thresh > 0.0:
            syn_df_1 = syn_df_1[
                syn_df_1[cleft_score_column_name] >= float(cleft_thresh)
            ].reset_index(drop=True)
        # makes df of just the coordinates converted into viewer resolution #
        syn_coords_df_1 = pd.DataFrame(
            {
                "pre": [
                    [coord / res for coord, res in zip(point, viewer_res)]
                    for point in syn_df_1["pre_pt_position"]
                ],
                "post": [
                    [coord / res for coord, res in zip(point, viewer_res)]
                    for point in syn_df_1["post_pt_position"]
                ],
            }
        )
        # adds df of synapse coords to list for passing into statebuilder #
        anno_df_list.append(syn_coords_df_1)
        # gets count of number of synapses #
        syn_count_1 = len(syn_coords_df_1)

        # repeats the process for the second direction if both incoming and outgoing are requested #
        if syn_state == "both":
            syn_df_2 = client.materialize.query_table(
                synapse_table_name,
                filter_in_dict={syn_col_name_2: root_ids},
            )
            if cleft_thresh > 0.0:
                syn_df_2 = syn_df_2[
                    syn_df_2[cleft_score_column_name] >= float(cleft_thresh)
                ].reset_index(drop=True)
            syn_coords_df_2 = pd.DataFrame(
                {
                    "pre": [
                        [coord / res for coord, res in zip(point, viewer_res)]
                        for point in syn_df_2["pre_pt_position"]
                    ],
                    "post": [
                        [coord / res for coord, res in zip(point, viewer_res)]
                        for point in syn_df_2["post_pt_position"]
                    ],
                }
            )
            anno_df_list.append(syn_coords_df_2)
            syn_count_2 = len(syn_coords_df_2)

    # makes empty list to fill with layers #
    layer_list = []

    # sets configuration for EM layer #
    img = ImageLayerConfig(
        name="EM",
        source=client.info.image_source(),
    )

    # adds EM layer to list #
    layer_list.append(img)

    seg_source = client.info.segmentation_source()
    if datastack == "brain_and_nerve_cord":
        split_url = seg_source.split("https")
        seg_source = split_url[0] + "middleauth+https" + split_url[1]

    # determines color scheme for segmentation #
    if white == True:
        color_list = ["#ffffff" for x in root_ids]
    elif custom_colors != False:
        color_list = custom_colors
    else:
        color_list = tt.generate_color_list(len(root_ids), alternate_brightness=0.3)

    # defines segmentation layer config #
    seg = SegmentationLayerConfig(
        name="Segmentation",
        source=seg_source,
        fixed_ids=root_ids,
        fixed_id_colors=color_list,
    )

    # adds segmentation layer to list #
    layer_list.append(seg)

    # creates empty list to fill with annotation layers
    anno_layers = []

    # defines synapse layer config(s) if requested #
    if syn_state != "none":
        # defines configuration for line annotations #
        lines = LineMapper(
            point_column_a="pre",
            point_column_b="post",
        )

        if datastack == "flywire_fafb_production":
            in_syn_color = "#FFFF00"
            out_syn_color = "#0000FF"
        else:
            in_syn_color = "#FF6E4A"
            out_syn_color = "#3D81FF"

        # ensures consistent color coding for incoming and outgoing synapses #
        if syn_state == "out":
            syn_1 = AnnotationLayerConfig(
                name=syn_tab_name_1,
                mapping_rules=lines,
                color=out_syn_color,
            )
        else:
            syn_1 = AnnotationLayerConfig(
                name=syn_tab_name_1,
                mapping_rules=lines,
                color=in_syn_color,
            )
        # layer_list.append(syn_1)
        anno_layers.append(syn_1)
        # repeats process if in and output synapses requested #
        if syn_state == "both":
            syn_2 = AnnotationLayerConfig(
                name=syn_tab_name_2,
                mapping_rules=lines,
                color=out_syn_color,
            )
            anno_layers.append(syn_2)

    # VIEW SET, CURRENTLY NONFUNCTIONAL #
    if datastack == "flywire_fafb_production":
        view_options = {
            "position": [130900, 57662, 3052],
            "zoom_3d": 6.64,
        }
    elif datastack == "brain_and_nerve_cord":
        view_options = {
            "position": [
                127339,
                142245,
                3041,
            ],
            # "zoom_3d": 6.64,
        }

    # adds bounding box layer if user requests it #
    if bbox_coord_list != False:
        
        # makes df out of coorindate list #
        bbox_df = pd.DataFrame(
            {
                "point_a": [bbox[0] for bbox in bbox_coord_list],
                "point_b": [bbox[1] for bbox in bbox_coord_list],
            }
        )

        anno_df_list.append(bbox_df)

        print(bbox_df)
        
        # defines configuration for bbox annotations #
        bbox_rules = BoundingBoxMapper(
            point_column_a="point_a",
            point_column_b="point_b",
        )

        # conditionally sets layer name plurality #
        if len(bbox_coord_list) == 1:
            bbox_layer_name = "Bounding Box"
        else:
            bbox_layer_name = "Bounding Boxes"

        # defines annotation layer for bbox #
        bbox_anno = AnnotationLayerConfig(
            name=bbox_layer_name,
            mapping_rules=bbox_rules,
        )

        anno_layers.append(bbox_anno)

###################################################################################

    # PARTIAL CODE FOR BBOX QUERY, CURRENTLY UNUSED #
    # ensures that all boxmin coords are less than their boxmax counterparts #
    # if boxmin[0] > boxmax[0]:
    #     temp = boxmax[0]
    #     boxmax[0] = boxmin[0]
    #     boxmin[0] = temp
    # if boxmin[1] > boxmax[1]:
    #     temp = boxmax[1]
    #     boxmax[1] = boxmin[1]
    #     boxmin[1] = temp
    # if boxmin[2] > boxmax[2]:
    #     temp = boxmax[2]
    #     boxmax[2] = boxmin[2]
    #     boxmin[2] = temp
    
    # # queries synapse table for synapses in bbox #
    # syn_df1 = client.materialize.query_table(
    #     "synapses_nt_v1",                
    #     filter_spatial_dict={"pre_pt_position": [boxmin,boxmax]},
    # )
    # syn_df2 = client.materialize.query_table(
    #     "synapses_nt_v1",                
    #     filter_spatial_dict={"post_pt_position": [boxmin,boxmax]},
    # )
    
    # # concatenates pre and post queried dfs #
    # syn_df = pd.concat([syn_df1,syn_df2],ignore_index=True)

    # # drops duplicate entires from df #
    # syn_df = syn_df.drop_duplicates(subset=['id'])

    # # makes df of just synapse coords #
    # syn_coords_df = pd.DataFrame(
    #     {
    #         "pre": [nmToNG(x, res) for x in syn_df["pre_pt_position"]],
    #         "post": [nmToNG(x, res) for x in syn_df["post_pt_position"]],
    #     }
    # )

 ###################################################################################

    # chooses target site name based on datastack to handle pre-spelunker datasets #
    if datastack == "flywire_fafb_production":
        target_site_name = "seunglab"
    else:
        target_site_name = "spelunker"

    # counts number of annotation layers #
    anno_layer_num = len(anno_layers)

    # behavior if there are no annotation layers #
    if anno_layer_num == 0:
        
        # defines state builder by passing in rules for img, seg, and anno layers #
        sb_1 = StateBuilder(
            layer_list,
            resolution=viewer_res,
            # view_kws=view_options,
        )
        
        # builds state json #
        state_json = json.loads(
            sb_1.render_state(
                return_as="json",
                target_site=target_site_name,
            )
        )
    
    # behavior if there is exactly 1 annotation layer #
    elif anno_layer_num == 1:
        
        # adds annotation layer to layer list #
        layer_list.append(anno_layers[0])
        
        # creates statebuilder #
        sb_1 = StateBuilder(
            layer_list,
            resolution=viewer_res,
            # view_kws=view_options,
        )
        
        # builds state json with one annotation layer #
        state_json = json.loads(
            sb_1.render_state(
                anno_df_list[0],
                return_as="json",
                target_site=target_site_name,
            )
        )
    
    # behavior if there are 2 or more annotation layers #
    else:
        # makes empty statebuilder list to populate for chained statebuilder #
        sb_list = []

        #adds first annotation layer to layer list for first statebuilder #
        layer_list.append(anno_layers[0])

        # makes first statebuilder #
        sb_1 = StateBuilder(
            layer_list,
            resolution=viewer_res,
            # view_kws=view_options,
        )
        # adds first statebuilder to statebuilder list #
        sb_list.append(sb_1)

        # iterates over remaining layers, making statebuilders for each and adding to list #
        for layer in anno_layers[1:]:
            sb = StateBuilder([layer],resolution=viewer_res)
            sb_list.append(sb)

        # makes chained statebuilder out of individuals #
        chained_sb = ChainedStateBuilder(sb_list)

        # makes state json using chained statebuilder and list of annotation dfs #
        state_json = json.loads(
            chained_sb.render_state(
                anno_df_list,
                return_as="json",
                target_site=target_site_name,
            )
        )

    # manually sets viewer angle based on dataset #
    if datastack == "flywire_fafb_production":
        state_json["perspectiveOrientation"] = [
            -0.029998891055583954,
            -0.008902468718588352,
            -0.02068145014345646,
            -0.9992963075637817,
        ]
    elif datastack == "brain_and_nerve_cord":
        state_json["perspectiveOrientation"] = [
            -0.006392207462340593,
            -0.9995864033699036,
            -0.0023922761902213097,
            -0.02793547324836254,
        ]

    # sends JSON state to remote state server and gets back state ID #
    new_state_id = client.state.upload_state_json(state_json)

    # builds url using the state ID and the base url for the dataset #
    url = client.state.build_neuroglancer_url(
        state_id=new_state_id,
        ngl_url=base_url,
    )

    # conditional synapse print based on user input #
    if syn_state == "in":
        print("Incoming Synapses:", syn_count_1)
    elif syn_state == "out":
        print("Outgoing Synapses:", syn_count_1)
    elif syn_state == "both":
        print("Incoming Synapses:", syn_count_1)
        print("Outgoing Synapses:", syn_count_2)
    
    return url

In [ ]:
build_ng_link([720575940625342472,720575940627818499,720575940631449916,720575940651305718,720575940618680550,720575940626405690], "flywire_fafb_production", incoming=False, outgoing=True,bbox_coord_list=[[[112521, 41993, 1595],[155287, 57562, 3349]],[[124387, 48074, 2093],[140105, 68117, 2994]],[[148352, 65261, 3343],[150240, 66971, 3093]]])

In [ ]:
from osteoid import Skeleton, Bbox
from caveclient import CAVEclient
# from cloudvolume import CloudVolume
import numpy as np
import microviewer
import time
import sys
import gspread
import pandas as pd

def check_root_intersect(root_list, datastack="brain_and_nerve_cord",visualize=False):
	client = CAVEclient(datastack)

	if datastack == "brain_and_nerve_cord":
		bbox_list = [
			Bbox([86240, 24188, 1504],[100628, 41809, 2229]), # tunnel of death #
			Bbox([85969, 34825, 2230],[100310, 44999, 2959]), # tunnel of death #
			Bbox([89471, 35922, 2935],[113945, 46013, 3691]), # tunnel of death #
			Bbox([113941, 38078, 3195],[126443, 48442, 3691]), # tunnel of death #
			Bbox([126446, 35586, 3145],[139404, 45760, 3691]), # tunnel of death #
			Bbox([139380, 35978, 3007],[150973, 42310, 3506]), # tunnel of death #
			Bbox([146357, 35253, 2605],[153882, 41070, 3005]), # tunnel of death #
			Bbox([149234, 34279, 2019],[153801, 39355, 2603]), # tunnel of death #
			Bbox([99778, 176438, 3251],[ 104086, 181975, 3687]), # T2 blowout #
			Bbox([146730, 194587, 4441],[148357, 198226, 4616]), # T1 Soup #
			Bbox([116523, 204075, 5478],[119830, 208313, 5620]), # champagne patch #
			Bbox([114396, 201794, 5620],[123629, 209989, 5882]), # champagne patch #
			Bbox([115172, 202544, 5885],[122619, 211033, 5975]), # champagne patch #
			Bbox([111512, 200317, 5975],[124825, 215415, 6385]), # champagne patch #
			Bbox([116261, 205698, 6385],[122632, 213127, 6495]), # champagne patch #     
		]

	pbbx_list = [bbox.convert_units("nm", resolution=[4,4,45]) for bbox in bbox_list]

	matrix = np.array([
	[16, 0, 0, 0],
	[0, 16, 0, 0],
	[0, 0, 45, 0],
	], dtype=np.float32)

	seconds = client.skeleton.generate_bulk_skeletons_async(root_list, skeleton_version=-1)

	if isinstance(seconds, dict):
		print("Bad root id.")
		sys.exit(0)

	print(f"ETA {seconds} seconds.")
	time.sleep(seconds)

	cskel_list = [client.skeleton.get_skeleton(i) for i in root_list]

	pskel_list = [Skeleton(vertices=c['vertices'], edges=c['edges'], radii=c['radius'], transform=matrix, space="physical") for c in cskel_list]

	if visualize == True:
		viewer_list = pskel_list + pbbx_list
		microviewer.objects(viewer_list)

	def check_skel(pskel, bbox_list):
		skel_intersect = False
		for box in bbox_list:
			crop_skel = pskel.crop(box)
			if not crop_skel.empty():
				skel_intersect = True
		return skel_intersect

	intersect_list = [check_skel(skel, pbbx_list) for skel in pskel_list]

	return intersect_list

In [ ]:
def check_ids_from_gsheet(sheet_key):
	gc = gspread.oauth()
	spreadsheet = gc.open_by_key(sheet_key)
	tab = spreadsheet.sheet1
	roots = tab.col_values(1)
	intersect_list = check_root_intersect(roots)

	df = pd.DataFrame({
		"root_ids":roots,
		"intersect_rough":intersect_list,
	})

	tab.update([df.columns.values.tolist()] + df.values.tolist())

In [ ]:
########################## CODE TO DIRECTLY QUERY CHANGELOG, CURRENTLY NONFUNCTIONAL ##################################

In [ ]:


def get_edits_json(datastack, start, end)

    def get_auth_token():
    """Find and return the local CAVE client auth token.

    Returns:
        auth_token: CAVEClient user authorization token (str)
    """
    f = open("authtoken.txt", "r")
    auth_token = f.read()
    f.close()
    return auth_token

    # sets client using datastack name #
    client = CAVEclient(datastack_name=datastack)

    # pulls datastact info dictionary using stack name #
    stack_info = client.info.get_datastack_info()

    cg_url = stack_info["segmentation_source"]
    # split_url = url.split("https")
    # chunkedgraph_url = split_url[0] + "middleauth+https" + split_url[1]

    # builds query url #
    url = f"{cg_url}/tabular_change_log_recent?{start_arg}={str(start)}&{end_arg}={str(end)}&middle_auth_token={auth_token}"

    # uses query url to send a request for all the edits to a given dataset in a given timeframe #
    r = requests.get(url)

    # converts the result of the request into json format #
    json = r.json()